# MongoDB Lesson2
After learning Lesson1, we start to test the effection of Learning.

# 1. Using csv module
key word: "csv.reader", "next()"

In [1]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""
import csv
import os

DATADIR = ""
DATAFILE = "745090.csv"


def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'r') as f:
        r = csv.reader(f, delimiter=',')
        #r = csv.DictReader(f) 
        cout = 0
        for line in r:      
            if cout == 0:
                name = line[1]
                print(name)
                cout += 1   
                
    with open(datafile,'r') as f:
        next(f) # skip header row
        next(f) 
        r = csv.reader(f, delimiter=',')
        for line in r: 
            data.append(line)
   
   
    # Do not change the line below
    return (name, data)

datafile = os.path.join(DATADIR, DATAFILE)
name, data = parse_file(datafile)
#print(header)
print("===============")
print(data[0][1])
print(data[2][0])
print(data[2][5])

def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"


if __name__ == "__main__":
    test()


MOUNTAIN VIEW MOFFETT FLD NAS
01:00
01/01/2005
2
MOUNTAIN VIEW MOFFETT FLD NAS


# 2. Excel to CSV 
一定要先檢查題目所要問的是甚麼, 並對照資料裡的內容. 比如說這次在做本題時, 並沒有看清楚題目沒有要求 "ERCOT" 欄位. 因此在對答案時出現錯誤. 

* 改善方式: 首先確定需要那些欄位.

In [71]:
# -*- coding: utf-8 -*-
'''
Find the time and value of max load for each of the regions
COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character | as the delimiter.

An example output can be seen in the "example.csv" file.
'''

import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    print(sheet.ncols)
    data = {}
    #data = [[sheet.cell_value(r, col)
     #           for col in range(sheet.ncols)]  # columns first
      #                 for r in range(sheet.nrows)] # rows second
    
    
    datetime = sheet.col_values(0, start_rowx = 1, end_rowx = sheet.nrows) 
    station = sheet.row_values(0) # to read a row on the top.
    print(station)
    col_value = 1
    for var in station:
        if col_value < sheet.ncols-1:   # exclude "ERCOT"
            var = sheet.col_values(col_value, start_rowx = 1, end_rowx = sheet.nrows)
            var_index = var.index(max(var))
            realtime = xlrd.xldate_as_tuple(datetime[var_index], 0)
            #print(station[col_value], max(var), realtime)
            data[station[col_value]] = {"maxval" : max(var), "maxtime": realtime}
            col_value += 1
        else:
            pass
    
    #print('======================')
    #COAST = sheet.col_values(1, start_rowx=1, end_rowx=sheet.nrows)
    #print(COAST.index(max(COAST)))
    
    # YOUR CODE HERE
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)
    return data # create dictionary on data

#open_zip(datafile)
data = parse_file(datafile)

print(data) 

def save_file(data, filename):
    with open(filename, "w") as f:
        w = csv.writer(f, delimiter="|")
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
        for s in data:
            year, month, day, hour, _, _ = data[s]["maxtime"]
            w.writerow([s, year, month, day, hour, data[s]["maxval"]])
    # YOUR CODE HERE
    
save_file(data, "2013_Max_Loads.csv")
    


10
['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST', 'ERCOT']
{'COAST': {'maxval': 18779.025510000003, 'maxtime': (2013, 8, 13, 17, 0, 0)}, 'EAST': {'maxval': 2380.1654089999956, 'maxtime': (2013, 8, 5, 17, 0, 0)}, 'FAR_WEST': {'maxval': 2281.2722140000024, 'maxtime': (2013, 6, 26, 17, 0, 0)}, 'NORTH': {'maxval': 1544.7707140000005, 'maxtime': (2013, 8, 7, 17, 0, 0)}, 'NORTH_C': {'maxval': 24415.570226999993, 'maxtime': (2013, 8, 7, 18, 0, 0)}, 'SOUTHERN': {'maxval': 5494.157645, 'maxtime': (2013, 8, 8, 16, 0, 0)}, 'SOUTH_C': {'maxval': 11433.30491600001, 'maxtime': (2013, 8, 8, 18, 0, 0)}, 'WEST': {'maxval': 1862.6137649999998, 'maxtime': (2013, 8, 7, 17, 0, 0)}, 'ERCOT': {'maxval': 67595.81819800012, 'maxtime': (2013, 8, 7, 17, 0, 0)}}


## \#另解

In [72]:
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = {}
    # process all rows that contain station data
    for n in range (1, 9):
        station = sheet.cell_value(0, n)
        cv = sheet.col_values(n, start_rowx=1, end_rowx=None)

        maxval = max(cv)
        maxpos = cv.index(maxval) + 1
        maxtime = sheet.cell_value(maxpos, 0)
        realtime = xlrd.xldate_as_tuple(maxtime, 0)
        data[station] = {"maxval": maxval,
                         "maxtime": realtime}

    print data
    return data

def save_file(data, filename):
    with open(filename, "w") as f:
        w = csv.writer(f, delimiter='|')
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
        for s in data:
            year, month, day, hour, _ , _= data[s]["maxtime"]
            w.writerow([s, year, month, day, hour, data[s]["maxval"]])

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(data)? (<ipython-input-72-a938b1c058f2>, line 17)